In [1]:
pip install matplotlib

Note: you may need to restart the kernel to use updated packages.


In [4]:
import itertools         #用于处理迭代器和循环的函数。
import numpy as np
import pandas as pd
import torch
import torch.nn as nn     #导入PyTorch中的nn模块，其中包含了构建神经网络的各种层和损失函数。
import torch.nn.functional as F   #导入PyTorch中的functional模块，其中包含了一些常用的函数，如激活函数和损失函数。
from torch import optim
from torch.utils.data import Dataset, DataLoader, TensorDataset#导入PyTorch中用于处理数据的类和函数，如数据集类Dataset、数据加载器类DataLoader和Tensor数据集类TensorDataset。
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss, roc_auc_score, accuracy_score, precision_score, f1_score
from sklearn.preprocessing import LabelEncoder, MinMaxScaler#如标签编码和最小最大缩放。
from collections import OrderedDict, namedtuple, defaultdict
import os
# os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
# %%capture

def get_metrics(loader, model):
    pred, target = [], []  # 初始化预测结果和真实标签的列表
    model.eval()  # 将模型设置为评估模式

    with torch.no_grad():
        for x, y in loader:
            x, y = x.to(device).float(), y.to(device).float()
            y_hat = model(x)
            # print(y_hat)
            # pred.append(y_hat.cpu().numpy())
            # target.append(y.cpu().numpy())
             
            pred += list(y_hat.cpu().numpy())
            target += list(y.cpu().numpy())

    # 合并为单个NumPy数组
    target = np.concatenate(target)
    pred = np.concatenate(pred)
    # print(pred[:5],len(pred))

    # 转换为PyTorch张量
    # target = torch.tensor(target)
    # pred = torch.tensor(pred)
    
    logloss = log_loss(target, pred)  # 计算 log loss
    auc = roc_auc_score(target, pred)  # 计算 AUC
    return logloss,auc
    
class DNN(nn.Module):
    def __init__(self, inputs_dim, hidden_units, dropout_rate,):
        super(DNN, self).__init__()
        self.inputs_dim = inputs_dim#输入的特征维度大小。
        self.hidden_units = hidden_units
        self.dropout = nn.Dropout(dropout_rate)#表示 DNN 中的 Dropout 层的丢弃率。
        self.hidden_units = [inputs_dim] + list(self.hidden_units)#其中包含输入维度和隐藏层神经元数量

        '''
            self.linear 是一个 nn.ModuleList 类型的列表。在这个列表中，存放了多个线性层（nn.Linear）模块，
        每个线性层都是神经网络的一层。nn.ModuleList 是 PyTorch 提供的一种容器类型，用于管理多个模块，它
        类似于 Python 中的列表，但具有一些额外的功能。
            通过使用列表解析表达式，我们创建了一个包含多个线性层的列表，并将这个列表存储在 self.linear 中。列表
        中的每个元素都是一个 nn.Linear 对象，代表着神经网络中的一层。
           这样设计的目的是方便在模型的前向传播过程中，依次遍历每一层，对输入进行线性变换和激活操作，构建深度神经网络
        的结构。由于 nn.ModuleList 中的模块会自动被注册为模型的子模块，因此在模型的训练和保存过程中，这些子模块
        也会被正确地处理。
           nn.Linear 对象中的权重和偏置可以通过 named_parameters() 方法或者 parameters() 方法来访问。这些
        方法会返回一个迭代器，其中包含了模型中的所有参数，每个参数都是一个 (name, parameter) 的元组，其中 name
        是参数的名称，parameter 是参数的值。
        '''
        self.linear = nn.ModuleList([
            nn.Linear(self.hidden_units[i], self.hidden_units[i + 1]) for i in range(len(self.hidden_units) - 1)
        ])
        for name, tensor in self.linear.named_parameters():
            if 'weight' in name:#"weight" 参数用于存储权重矩阵
                nn.init.normal_(tensor, mean=0, std=0.0001)

        self.activation = nn.ReLU()#用于在每个隐藏层之后引入非线性性。

    def forward(self, X):#使用模型实例进行预测或训练时，会调用 forward 方法。
        inputs = X
        for i in range(len(self.linear)):
            fc = self.linear[i](inputs)
            fc = self.activation(fc)
            fc = self.dropout(fc)
            inputs = fc
        return inputs

# class SENETLayer(nn.Module):
#     def __init__(self, filed_size, reduction_ratio=3):
#         super(SENETLayer, self).__init__()
#         #计算 SE 模块中的通道缩减后的维度大小。其中，filed_size 为输入稀疏特征的维度大小，reduction_ratio 为缩减比例。max(1, filed_size // reduction_ratio) 确保通道缩减后的维度至少为 1。
#         #filed_size 表示输入稀疏特征的维度大小。具体来说，它指的是 SE 模块的输入特征的数量，也就是稀疏特征的个数。
#         # filed_size 除以 reduction_ratio 并向下取整，得到通道缩减后的维度大小。
#         self.reduction_size = max(1, filed_size // reduction_ratio)
#         self.excitation = nn.Sequential(
#             nn.Linear(filed_size, self.reduction_size, bias=False),
#             nn.ReLU(),
#             nn.Linear(self.reduction_size, filed_size, bias=False),
#             nn.ReLU(),
#         )

#     '''
#     在模型的前向传播过程中，PyTorch 会自动调用 forward 方法，以完成对输入数据的处理和模型的前向计算。
#     '''
#     def forward(self, inputs):
#         # inputs [1024, 26, 4]
#         # Z [1024, 26]
#         '''
# inputs: 这是一个形状为 [batch_size, filed_size, embedding_dim] 的输入张量。其中，batch_size 表示输入数据的批量大小，
#         filed_size 表示输入稀疏特征的维度（稀疏特征的数量），embedding_dim 表示稀疏特征的嵌入维度。
# dim=-1: 这是 torch.mean() 函数的参数，表示在哪个维度上求平均值。
#         在这里，dim=-1 表示在最后一个维度（即 embedding_dim 维度）上求平均值。这样，对于每个输入特征，会计算其嵌入维度的平均值，得到形状为
#          [batch_size, filed_size] 的张量 Z。
# out=None: 这也是 torch.mean() 函数的参数，表示输出的张量。如果 out 不为 None，则结果会被存储在指定的张量中；否则，将返回一个新的张量。在这里，
#          没有指定 out，因此将返回一个新的张量 Z，其中包含每个输入特征在 embedding_dim 维度上的平均值。
# 综上所述，Z 是通过对输入张量 inputs 在 embedding_dim 维度上求平均值得到的结果，形状为 [batch_size, filed_size]，即每个特征在嵌入维度上的平均值。
#         '''
#         Z = torch.mean(inputs, dim=-1, out=None)
#         # A [1024, 26]
#         A = self.excitation(Z)
#         '''
#         torch.unsqueeze(A, dim=2): 这个函数在张量A中添加了一个新的维度。新的维度插入在索引2处，这意味着A的形状从(batch_size, filed_size)变
#         为(batch_size, filed_size, 1)。这样做是为了使inputs和A的形状在逐元素相乘时兼容。
#         torch.mul(inputs, torch.unsqueeze(A, dim=2)): 这个表达式对inputs和A进行逐元素相乘。由于两个张量的维度数相同，PyTorch会自动进行广播
#         （broadcasting），使得相乘在相应的维度上逐元素进行。
#         '''
#         V = torch.mul(inputs, torch.unsqueeze(A, dim=2))
#         return V

# class BilinearInteraction(nn.Module):
#     def __init__(self, filed_size, embedding_size, bilinear_type='each'):
#         super(BilinearInteraction, self).__init__()
#         self.bilinear_type = bilinear_type
#         self.bilinear = nn.ModuleList()
#         if self.bilinear_type == 'all':
#             self.bilinear = nn.Linear(embedding_size, embedding_size,bias=False)
#         elif self.bilinear_type == 'each':
#             for _ in range(filed_size):
#                 self.bilinear.append(nn.Linear(embedding_size, embedding_size, bias=False))
#         elif self.bilinear_type == 'interaction':
#             for _, _ in itertools.combinations(range(filed_size), 2):
#                 self.bilinear.append(nn.Linear(embedding_size, embedding_size, bias=False))

#     def forward(self, inputs):
#         '''
#         通过设置 split_size_or_sections=1 和 dim=1，inputs 张量将被沿着 dim=1 维度分割成 num_features 个较小的张量，
#         每个张量将包含沿着 dim=1 维度的一个元素。每个分割张量表示批次中所有样本的一个稀疏特征的嵌入。
#         分割操作后，inputs 将成为一个张量列表，其中每个张量的形状为 (batch_size, 1, embedding_size)

#         itertools.combinations 函数用于生成给定迭代器中所有长度为 2 的组合。这是一个在组合数学中常用的操作，它可以得到从给定集合中选取 2 个元素的所有可能组合。
#         :param inputs:
#         :return:
#         '''
#         inputs = torch.split(inputs, 1, dim=1)
#         if self.bilinear_type == 'all':
#             p = [torch.mul(self.bilinear(v_i), v_j) for v_i, v_j in itertools.combinations(inputs, 2)]
#         elif self.bilinear_type == 'each':
#             p = [torch.mul(self.bilinear[i](inputs[i]), inputs[j]) for i, j in itertools.combinations(range(len(inputs)), 2)]
#         elif self.bilinear_type == 'interaction':
#             p = [torch.mul(bilinear(v[0]), v[1]) for v, bilinear in zip(itertools.combinations(inputs, 2), self.bilinear)]
#         return torch.cat(p, dim=1)

#     '''
#         feat_size: feat_size 是一个字典，用于存储每个特征（feature）的可能取值数量。它包含两类特征：稠密特征（dense features）和稀疏特征
#     （sparse features）。字典的键是特征的名称，值是对应特征可能取值的数量。例如，如果有一个名为 "age" 的稠密特征，其可能取值范围是 0 到 100.
#     那么 feat_size['age'] 就等于 101（即包含 101 个可能的取值）。

#        filed_size: filed_size 是一个整数，表示输入稀疏特征的维度大小。在上下文中，filed_size 指的是 SE 模块的输入特征数量，即稀疏特征的个数。

#        embedding_dim: embedding_dim 是一个整数，表示嵌入向量的维度大小。在模型中，稀疏特征会被转换为嵌入向量，这样可以将高维稀疏特征表示为低维稠密向量，
#     从而减少参数量并且更好地表示特征之间的关系。embedding_dim 定义了嵌入向量的维度大小，通常是一个模型超参数，需要根据具体任务和数据集进行调整。
#     '''

# class CrossNet(nn.Module):
#     def __init__(self, in_features, layer_num=3, parameterization='vector', seed=2022):
#         super(CrossNet, self).__init__()
#         self.layer_num = layer_num
#         self.parameterization = parameterization  # 向量化的参数化方式
#         if self.parameterization == 'vector':
#             self.kernels = nn.Parameter(torch.Tensor(self.layer_num, in_features, 1))
#         elif self.parameterization == 'matrix':
#             self.kernels = nn.Parameter(torch.Tensor(self.layer_num, in_features, in_features))
#         self.bias = nn.Parameter(torch.Tensor(self.layer_num, in_features, 1))

#         for i in range(self.kernels.shape[0]):
#             nn.init.xavier_normal_(self.kernels[i])
#         for i in range(self.bias.shape[0]):
#             nn.init.zeros_(self.bias[0])  # 将第一层的偏置初始化为零

#     def forward(self, inputs):
#         x_0 = inputs.unsqueeze(2)  # 在指定维度上添加一个维度
#         x_1 = x_0
#         for i in range(self.layer_num):
#             if self.parameterization == 'vector':
#                 x1_w = torch.tensordot(x_1, self.kernels[i], dims=([1], [0]))  # torch.tensordot 是一个用于计算张量积的函数
#                 dot_ = torch.matmul(x_0, x1_w)
#                 x_1 = dot_ + self.bias[i] + x_1
#             else:
#                 x1_w = torch.tensordot(self.kernels[i], x_1)
#                 dot_ = x1_w + self.bias[i]
#                 x_1 = x_0 * dot_ + x_1
#         x_1 = torch.squeeze(x_1, dim=2)
#         return x_1
class  FiBiNET(nn.Module):
    def __init__(self, feat_size, embedding_size, linear_feature_columns, dnn_feature_columns,
                 dnn_hidden_units=(256,128), num_heads=2,l2_reg=1e-4, dnn_dropout=0.5,
                 reduction_ratio=3, bilinear_type='each',cross_num=3,cross_param='vector',init_std=0.0001,drop_rate=0.5):
        super(FiBiNET, self).__init__()
        self.act = nn.ReLU()
        self.dropout = nn.Dropout(drop_rate)
        #稀疏特征列，包含了模型的离散特征信息。根据指定的条件过滤一个可迭代对象，并返回符合条件的元素。
        self.sparse_feature_columns = list(filter(lambda x: x[1] == 'sparse', dnn_feature_columns))

        #存储离散特征的Embedding层，使用nn.Embedding创建。每个特征的Embedding大小为embedding_size。
        '''
           在这里，对于每个离散特征 feat，根据其特征大小 feat_size[feat[0]] 和指定的 embedding_size（嵌入向量的维度），
        创建一个嵌入层 nn.Embedding。然后将该嵌入层添加到 embedding_dic 字典中，并使用 feat[0]（特征名称）作为键。
        它将高维的离散特征映射到低维的连续向量空间中。
           self.embedding_dic 是一个包含所有稀疏特征嵌入层的 nn.ModuleDict 对象，可以通过特征名称（键）来获取对应的嵌入层（值）。
        在模型的前向传播过程中，将会使用这些嵌入层将稀疏特征转换为嵌入向量。
           sparse=False 表示稠密化嵌入向量。
        '''
        self.embedding_dic = nn.ModuleDict({
            feat[0]: nn.Embedding(feat_size[feat[0]], embedding_size, sparse=False) for feat in self.sparse_feature_columns
        })
        self.dense_feature_columns = list(filter(lambda x: x[1] == 'dense', dnn_feature_columns))
        '''
        在这段代码中，self.feature_index 是一个默认值为整数 0 的字典。这样，在字典中查询一个不存在的键时，会自动返回默认值 0 而不会报错。
        '''
        self.feature_index = defaultdict(int)
        #为特征名称（feat）建立索引位置（整数）的映射关系，并从 0 开始递增，确保不同特征拥有不同的索引位置。
        start = 0
        for feat in feat_size:
            self.feature_index[feat] = start
            start += 1

#         # 引入多头注意力机制
#         self.multi_head_attention = nn.MultiheadAttention(embed_dim=embedding_size, num_heads=num_heads)

#         self.filed_size = len(self.embedding_dic)#特征数目，即输入数据中的总特征个数。
#         self.SE = SENETLayer(self.filed_size, reduction_ratio)#SENET层，用于学习特征之间的权重。
#         self.Bilinear = BilinearInteraction(self.filed_size, embedding_size)#BilinearInteraction层，用于进行特征交叉操作。
        #计算了 FiBiNET 模型中 DNN 层的输入维度。它由两部分组成：稀疏特征的交互部分和密集特征部分。
        '''
        这段代码中，计算了所有稀疏特征之间两两交叉的组合数，并且每个组合产生 embedding_size 维的交叉特征。
           假设 self.filed_size = n，那么共有 n * (n - 1) 种两两交叉的组合。每种组合生成 embedding_size 维的特征，
        因此特征交叉后的维度大小为 n * (n - 1) * embedding_size。
        '''
        # dim = self.filed_size * (self.filed_size-1)*embedding_size + len(self.dense_feature_columns)  #???
        dim = len(self.dense_feature_columns)+embedding_size*len(self.sparse_feature_columns)
        self.dnn = DNN(dim, dnn_hidden_units, 0.5)
        # self.dnn_linear = nn.Linear(dnn_hidden_units[-1], 1, bias=False)#用于将 DNN 的输出转换为单个输出节点。1 表示输出维度为 1，因为这是一个二分类问题，输出是一个概率值。
        # self.crossnet = CrossNet(dim, cross_num, cross_param)
        # self.dnn = DNN(dim, dnn_hidden_units, 0.5)
        self.dnn_linear = nn.Linear( dnn_hidden_units[-1], 1, bias=False)
        # self.dnn_linear = nn.Linear(dim + dnn_hidden_units[-1], 1, bias=False)

        # dnn_hidden_units = [len(feat_size)] + list(dnn_hidden_units) + [1]
        # self.linear = nn.ModuleList([
        #     nn.Linear(dnn_hidden_units[i], dnn_hidden_units[i + 1]) for i in range(len(dnn_hidden_units) - 1)
        # ])
        # for name, tensor in self.linear.named_parameters():
        #     if 'weight' in name:
        #         nn.init.normal_(tensor, mean=0, std=init_std)

    def forward(self, X):# FiBiNET 模型的前向传播（Forward Propagation）部分。

        # logit = X
        # for i in range(len(self.linear)):
        #     fc = self.linear[i](logit)#在循环中，对当前隐藏层 self.linear[i] 应用线性变换，即进行矩阵乘法。
        #     fc = self.act(fc)
        #     fc = self.dropout(fc)
        #     logit = fc#将处理后的结果 fc 赋值回 logit 变量，作为下一层的输入。

        sparse_embedding = [
            self.embedding_dic[feat[0]](X[:, self.feature_index[feat[0]]].long()).reshape(X.shape[0], 1, -1)
            for feat in self.sparse_feature_columns]#创建了一个列表 sparse_embedding，用于存储所有离散特征的嵌入向量。
        '''
        表示将列表 sparse_embedding 中的所有张量在维度 dim=1 上进行拼接。
        由于每个嵌入向量的第二个维度为 1，拼接时相当于去掉这个维度，将所有嵌入向量的第一维度拼接在一起。
        '''
        sparse_input = torch.cat(sparse_embedding, dim=1)
        # 拉直 本来是 [batch_size, sparase特征数, 嵌入维度] =》 [batch_size, sparase特征数 * 嵌入维度]
        sparse_input = torch.flatten(sparse_input, start_dim=1)
        # self.multi_head_attention 函数对 sparse_input 进行多头自注意力计算，用于学习特征之间的交互。
        # multi_head_output, _ = self.multi_head_attention(sparse_input, sparse_input, sparse_input)

        dense_values = [X[:, self.feature_index[feat[0]]].reshape(-1, 1) for feat in self.dense_feature_columns]
        dense_input = torch.cat(dense_values, dim=1)

#         senet_output = self.SE(sparse_input) # [1024, 26, 4]
#         senet_bilinear_out = self.Bilinear(senet_output) # [1024, 325, 4]
#         # bilinear_out = self.Bilinear(sparse_input) # [1024, 325, 4]
#         bilinear_out = self.Bilinear(multi_head_output)  # [1024, 325, 4]
#         dnn_input = torch.flatten(torch.cat((senet_bilinear_out, bilinear_out), dim=1), start_dim=1) # [1024, 2600]
        # dnn_input = torch.flatten(torch.cat(( bilinear_out,), dim=1), start_dim=1) # [1024, 2600]
        dnn_input = torch.cat((sparse_input, dense_input), dim=1) # [1024, 2613]
        dnn_out = self.dnn(dnn_input) # [1024, 128]
        # cross_out = self.crossnet(dnn_input)
        # stack_out = torch.cat((cross_out, dnn_out), dim=-1)
        final_logit = self.dnn_linear(dnn_out)
        # dnn_logit = self.dnn_linear(dnn_out) # [1024, 1]
        # final_logit = logit + dnn_logit
        y_pred = torch.sigmoid(final_logit)
        return y_pred

if __name__ == '__main__':

    batch_size =500
    lr = 1e-3
    wd = 1e-5
    epoches = 25
    seed = 2022
    embedding_size =20
    #device = 'cuda:0'
    device = 'cpu'
    # pd.set_option('display.max_rows', None)  # 显示数据中所有的列
    data = pd.read_csv('vehicle_data_model_50w_3.csv')
    dense_feature= ['driver_auth_success_days','cargo_search_cnt_3','cargo_search_cnt_7','scan_cargo_cnt_3','scan_cargo_cnt_7','click_cargo_cnt_3_x','click_cargo_cnt_7','call_cnt_3_driver','call_cnt_7_driver',
               'shipper_auth_success_days','exposure_cargo_cnt_3','exposure_cnt_3','click_cargo_cnt_3_y','click_cnt_3','cargo_weight','vector_regular_subscribe_line',
                       'vector_regular_cargo_line_all','vector_regular_cargo_truck_type_all','vector_regular_cargo_truck_length_all','vector_regular_cargo_line_30',
                       'vector_regular_cargo_truck_type_30','vector_regular_cargo_truck_length_30']

    # 假设你的数据集中包含'label'列，并且dense_feature已经定义
    sparse_feature  = data.drop(columns=['label'] + dense_feature).columns.tolist()
    print(len(sparse_feature))
    print('数据维度:',data.shape,'标签比例:',data['label'].value_counts())
    pd.options.display.max_rows = None  # 显示所有列
    data[sparse_feature]=data[sparse_feature].astype('uint8')
    target = ['label']

    feat_sizes = {}  # 初始化一个空字典 feat_sizes。
    feat_sizes_dense = {feat: 1 for feat in dense_feature}#这里将稠密特征的维度大小设置为1，因为这些特征不需要经过 Embedding 层，直接作为输入。
    # 对每个稀疏特征创建一个键值对，键为特征名称，值为该特征在数据中唯一取值的数量（即不同的类别个数）。
    feat_sizes_sparse = {feat: len(data[feat].unique()) for feat in sparse_feature}
    # 将稠密特征和稀疏特征的维度大小更新到 feat_sizes 字典中，得到包含所有特征维度大小信息的字典 feat_sizes。
    feat_sizes.update(feat_sizes_dense)
    feat_sizes.update(feat_sizes_sparse)



    # 定义fixlen_feature_columns，包含了所有特征的名称和类型（sparse或dense）。
    fixlen_feature_columns = [(feat, 'sparse') for feat in sparse_feature] + [(feat, 'dense') for feat in dense_feature]
    dnn_feature_columns = fixlen_feature_columns
    linear_feature_columns = fixlen_feature_columns

     # 数据集划分
    train, test = train_test_split(data, test_size=0.2, random_state=seed)
    validation, test = train_test_split(test, test_size=0.5, random_state=seed)
    
    # 计算每个数据集中 0 和 1 标签的数量
    train_label_summary = train['label'].value_counts()
    validation_label_summary = validation['label'].value_counts()
    test_label_summary = test['label'].value_counts()

    # 输出结果
    print("训练集标签汇总:\n", train_label_summary)
    print("验证集标签汇总:\n", validation_label_summary)
    print("测试集标签汇总:\n", test_label_summary)

    # DataLoader准备
    def create_data_loader(df, batch_size):
        labels = pd.DataFrame(df['label'])
        features = df.drop(columns=['label'])
        tensor_data = TensorDataset(torch.from_numpy(np.array(features)), torch.from_numpy(np.array(labels)))
        return DataLoader(tensor_data, shuffle=True, batch_size=batch_size)


    train_loader = create_data_loader(train, batch_size)
    validation_loader = create_data_loader(validation, batch_size)
    test_loader = create_data_loader(test, batch_size)

    # 模型初始化
    # model = FiBiNET(feat_sizes, embedding_size, linear_feature_columns, dnn_feature_columns)
    #device = 'cuda:0'
    device = 'cpu'
    model = FiBiNET(feat_sizes, embedding_size, linear_feature_columns, dnn_feature_columns).to(device)
    loss_func = nn.BCELoss(reduction='mean')
    optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=wd)
    # 早停策略参数
    early_stopping_threshold = 0.798
    best_validation_auc = 0


    # 训练循环
    for epoch in range(epoches):
        total_loss_epoch = 0.0
        total_tmp = 0
        model.train()
        for index, (x, y) in enumerate(train_loader):
            x, y = x.to(device).float(), y.to(device).float()
            y_hat = model(x)

            optimizer.zero_grad()
            loss = loss_func(y_hat, y)
            loss.backward()
            optimizer.step()
            total_loss_epoch += loss.item()
            total_tmp += 1

       # 验证集评估
        validation_loss, validation_auc = get_metrics(validation_loader,model)
        print(
            f'Epoch {epoch}/{epoches}, Train Loss: {total_loss_epoch / total_tmp:.4f}, Validation_loss: {validation_loss:.4f},Validation AUC: {validation_auc:.4f}')

        # 更新最佳验证集AUC
        if validation_auc > best_validation_auc:
            best_validation_auc = validation_auc

        # 早停判断
        if validation_auc >= early_stopping_threshold:
            print(f'Early stopping triggered at epoch {epoch}, Validation AUC: {validation_auc:.4f}')
            break

    # 测试集评估
    test_loss, final_test_auc = get_metrics(test_loader,model)
    print(f'Test_loss:{test_loss:.4f},Final Test AUC: {final_test_auc:.4f}')

17
数据维度: (500000, 40) 标签比例: label
0    467495
1     32505
Name: count, dtype: int64
训练集标签汇总:
 label
0    374034
1     25966
Name: count, dtype: int64
验证集标签汇总:
 label
0    46713
1     3287
Name: count, dtype: int64
测试集标签汇总:
 label
0    46748
1     3252
Name: count, dtype: int64
Epoch 0/25, Train Loss: 0.2453, Validation_loss: 0.2262,Validation AUC: 0.7034
Epoch 1/25, Train Loss: 0.2306, Validation_loss: 0.2216,Validation AUC: 0.7262
Epoch 2/25, Train Loss: 0.2258, Validation_loss: 0.2180,Validation AUC: 0.7440
Epoch 3/25, Train Loss: 0.2229, Validation_loss: 0.2157,Validation AUC: 0.7538
Epoch 4/25, Train Loss: 0.2206, Validation_loss: 0.2149,Validation AUC: 0.7586
Epoch 5/25, Train Loss: 0.2186, Validation_loss: 0.2138,Validation AUC: 0.7630
Epoch 6/25, Train Loss: 0.2164, Validation_loss: 0.2127,Validation AUC: 0.7666
Epoch 7/25, Train Loss: 0.2148, Validation_loss: 0.2119,Validation AUC: 0.7699
Epoch 8/25, Train Loss: 0.2133, Validation_loss: 0.2104,Validation AUC: 0.7728
Epoch 9/25,